In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
import os
from PIL import Image
import numpy as np

# Ana veri seti klasörünüz
dataset_dir = "dataset"
haze_dir = os.path.join(dataset_dir, "haze")
clear_dir = os.path.join(dataset_dir, "clear")


In [41]:
from google.colab import drive
import os
import matplotlib.pyplot as plt
from PIL import Image

# dataset klasörünü bağlama
dataset_path = '/content/drive/MyDrive/dataset'
#os.symlink(dataset_path, '/content/dataset')  # Sembolik bağlantı
working_dir = '/content/dataset'
# Alt klasörler
haze_dir = os.path.join(dataset_path, 'haze')
clear_dir = os.path.join(dataset_path, 'clear')

In [42]:
# Haze ve clear klasörlerindeki tüm görüntüleri listeleme
haze_images = [os.path.join(haze_dir, fname) for fname in os.listdir(haze_dir) if fname.endswith(".jpg")]
clear_images = [os.path.join(clear_dir, fname) for fname in os.listdir(clear_dir) if fname.endswith(".jpg")]

# İlk birkaç görüntüyü kontrol etmek için yükleyip gösterelim
for img_path in haze_images[:5]:
    img = Image.open(img_path)
    img.show()


In [43]:
import numpy as np
from PIL import Image
from multiprocessing import Pool

# Görüntüleri yükleme ve ön işleme fonksiyonu
def load_and_preprocess_image(img_path, target_size=(256, 256)):  # Boyutlandırmayı 256'e düşürdüm
    try:
        print(f"İşleniyor: {img_path}")  # Loglama eklendi
        img = Image.open(img_path).resize(target_size)  # Yeniden boyutlandırma
        img = np.array(img) / 255.0  # Normalizasyon
        return img
    except Exception as e:
        print(f"Görüntü yüklenemedi: {img_path}, Hata: {e}")
        return None

# Paralel işleme ile görüntüleri yükleme fonksiyonu
def load_images_parallel(image_paths, num_workers=4):
    with Pool(processes=num_workers) as pool:
        images = pool.map(load_and_preprocess_image, image_paths)

    # None dönen hatalı görüntüleri filtreleme
    images = [img for img in images if img is not None]
    return np.array(images)

# Alt küme ile test etmek isterseniz:
haze_images_subset = haze_images[:50]  # İlk 50 sisli görüntüyü al
clear_images_subset = clear_images[:50]  # İlk 50 temiz görüntüyü al

# Haze görüntüleri yükleme
print("Haze görüntüleri yükleniyor...")
haze_data = load_images_parallel(haze_images_subset, num_workers=4)  # 4 çekirdek kullanılıyor
print(f"Haze görüntüleri yüklendi, toplam {len(haze_data)} adet görüntü işleme alındı.")

# Clear görüntüleri yükleme
print("Clear görüntüleri yükleniyor...")
clear_data = load_images_parallel(clear_images_subset, num_workers=4)
print(f"Clear görüntüleri yüklendi, toplam {len(clear_data)} adet görüntü işleme alındı.")

# Yüklenen veri kümelerinin boyutlarını kontrol et
print(f"Haze veri şekli: {haze_data.shape}")
print(f"Clear veri şekli: {clear_data.shape}")



Haze görüntüleri yükleniyor...
İşleniyor: /content/drive/MyDrive/dataset/haze/8824_0.9_0.06.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8825_0.8_0.1.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8818_0.95_0.12.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8829_1_0.04.jpg



İşleniyor: /content/drive/MyDrive/dataset/haze/8829_0.8_0.06.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8823_0.95_0.06.jpg

İşleniyor: /content/drive/MyDrive/dataset/haze/8823_0.9_0.08.jpg
İşleniyor: /content/drive/MyDrive/dataset/haze/8825_0.9_0.2.jpg
İşleniyor: /content/drive/MyDrive/dataset/haze/8829_0.95_0.12.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8818_1_0.04.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8823_0.95_0.12.jpg


İşleniyor: /content/drive/MyDrive/dataset/haze/8827_0.8_0.04.jpg
İşleniyor: /content/drive/MyDrive/dataset/haze/8827_0.8_0.1.jpg
İşleniyor: /content/drive/MyDrive/dataset/haze/8827_0.95_0.06.jpgİşleniyor: /content/drive/MyDrive/dataset/haze/8818_0.8_0.12.j

In [44]:
import os

# Dataset yolunu kontrol et
dataset_path = '/content/drive/MyDrive/dataset'
clear_dir = os.path.join(dataset_path, "clear")
haze_dir = os.path.join(dataset_path, "haze")

# Klasörlerin varlığını doğrula
if not os.path.exists(clear_dir):
    raise FileNotFoundError(f"'clear' klasörü bulunamadı: {clear_dir}")
if not os.path.exists(haze_dir):
    raise FileNotFoundError(f"'haze' klasörü bulunamadı: {haze_dir}")

# Clear klasöründeki tüm dosyaların isimlerini alalım
clear_images = [fname for fname in os.listdir(clear_dir) if fname.endswith(".jpg")]

# Haze klasöründeki tüm dosyaların isimlerini alalım
haze_images = [fname for fname in os.listdir(haze_dir) if fname.endswith(".jpg")]

# Clear görüntüsüne karşılık gelen haze görüntülerini eşleştireceğimiz bir sözlük oluşturuyoruz
haze_images_dict = {clear_img: [] for clear_img in clear_images}

# Clear görüntülerine karşılık gelen haze görüntülerini bulalım
for clear_img in clear_images:
    # Clear görüntüsünün dosya adı (uzantısız)
    base_name = os.path.splitext(clear_img)[0]

    # Bu base_name'e sahip tüm haze dosyalarını buluyoruz
    corresponding_haze = [haze_img for haze_img in haze_images if haze_img.startswith(base_name)]

    # Eğer haze görüntüleri varsa, clear görüntüsüne karşılık gelen listeye ekliyoruz
    if corresponding_haze:
        for haze_img in corresponding_haze:
            haze_images_dict[clear_img].append(os.path.join(haze_dir, haze_img))

# Sonuçları kontrol edelim
for clear_img, haze_list in haze_images_dict.items():
    print(f"{clear_img} için {len(haze_list)} haze görüntüsü bulundu:")
    for haze_img in haze_list:
        print(f"    - {haze_img}")


Görüntülenen çıkış son 5000 satıra kısaltıldı.
    - /content/drive/MyDrive/dataset/haze/2487_0.8_0.16.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.8_0.2.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.8_0.1.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.95_0.1.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.95_0.06.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.85_0.12.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.85_0.06.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.95_0.2.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.85_0.16.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.9_0.08.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.95_0.04.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.9_0.12.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.95_0.12.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.9_0.06.jpg
    - /content/drive/MyDrive/dataset/haze/2487_1_0.2.jpg
    - /content/drive/MyDrive/dataset/haze/2487_0.9_0

## 0 haze resmi olan clear görüntüleri bulalım
zero_haze_clear_imagess = [clear_img for clear_img, haze_list in haze_images_dict.items() if len(haze_list) == 0]

# Sonuçları yazdıralım
if zero_haze_clear_images:
    print("0 haze resmi olan clear görüntüler:")
    for clear_img in zero_haze_clear_imagess:
        print(f"    - {clear_img}")
else:
    print("Tüm clear görüntüler için en az bir haze görüntüsü var.")

print(len(zero_haze_clear_imagess))

Haze'i bulunmayan clear data dosya adları: -2949.jpg - 2946.jpg - 2989.jpg - 2831.jpg - 2856.jpg - 2935.jpg - 2675.jpg - 2979.jpg - 2881.jpg - 2980.jpg - 2676.jpg - 2954.jpg - 2997.jpg - 2994.jpg - 2864.jpg - 2868.jpg - 2981.jpg - 2650.jpg - 2834.jpg - 2647.jpg - 2993.jpg - 2990.jpg - 2880.jpg - 2869.jpg - 2867.jpg - 2944.jpg - 2971.jpg - 2938.jpg - 2945.jpg - 2988.jpg - 2863.jpg - 2848.jpg - 2943.jpg - 2836.jpg - 2973.jpg - 2649.jpg - 2941.jpg - 2992.jpg - 2982.jpg - 2947.jpg - 2854.jpg - 2679.jpg - 2986.jpg - 2995.jpg - 2860.jpg - 2975.jpg - 2855.jpg - 2948.jpg - 2853.jpg - 2756.jpg - 2865.jpg - 2978.jpg - 2977.jpg - 2866.jpg - 2976.jpg - 2987.jpg - 2646.jpg - 2665.jpg - 2985.jpg - 2974.jpg - 2984.jpg - 2999.jpg - 2754.jpg - 2858.jpg - 2837.jpg - 2970.jpg - 2861.jpg - 2972.jpg - 2998.jpg - 3062.jpg - 3065.jpg - 3001.jpg - 3059.jpg - 3014.jpg - 3004.jpg - 3064.jpg - 3066.jpg - 3002.jpg - 3040.jpg - 3058.jpg - 3007.jpg - 3028.jpg - 3009.jpg - 3054.jpg - 3008.jpg

In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DynamicCNN(nn.Module):
    def __init__(self):
        super(DynamicCNN, self).__init__()
        # Evrişimsel katmanlar
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)

        # Upsampling: Giriş boyutunu geri getirmek için
        self.upsample = nn.Upsample(scale_factor=1, mode="bilinear", align_corners=True)

        # Çıkış katmanı
        self.output_layer = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)  # 3 kanal çıkış (RGB)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        # Giriş boyutuna geri döndür
        x = self.upsample(x)
        x = self.output_layer(x)
        return x



In [46]:
import os
from torch.utils.data import Dataset
from PIL import Image
import torch
import numpy as np

class HazeDataset(Dataset):
    def __init__(self, haze_dict, clear_dir, haze_dir, transform=None):
        """
        haze_dict: Haze görüntülerine karşılık gelen clear görüntüleri içeren sözlük
        clear_dir: Clear görüntülerin bulunduğu klasör yolu
        haze_dir: Haze görüntülerin bulunduğu klasör yolu
        transform: Görüntülere uygulanacak dönüşümler (PyTorch'da augmentasyon vb. için)
        """
        self.pairs = []  # Clear-Haze çiftlerini burada saklayacağız
        self.clear_dir = clear_dir
        self.haze_dir = haze_dir
        self.transform = transform

        # Sözlükten clear-haze çiftlerini oluştur
        for clear_img, haze_imgs in haze_dict.items():
            for haze_img in haze_imgs:
                self.pairs.append((clear_img, haze_img))

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        clear_img_name, haze_img_name = self.pairs[idx]

        # clear_img_name ve haze_img_name burada string olarak tutuluyor
        clear_path = os.path.join(self.clear_dir, clear_img_name)
        haze_path = os.path.join(self.haze_dir, haze_img_name)

        clear_image = Image.open(clear_path).convert("RGB")
        haze_image = Image.open(haze_path).convert("RGB")

        # Dönüşümleri uygula (eğer belirtilmişse)
        if self.transform:
            clear_image = self.transform(clear_image)
            haze_image = self.transform(haze_image)

        return clear_image, haze_image, clear_img_name  # clear_img_name'ı döndürüyoruz



In [47]:
from torchvision import transforms
from torch.utils.data import DataLoader

# Görüntü dönüşümleri (örnek augmentasyon)
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Boyutlandırma
    transforms.ToTensor()          # Tensöre çevirme
])

# Klasör yolları
clear_dir = "/content/drive/MyDrive/dataset/clear"
haze_dir = "/content/drive/MyDrive/dataset/haze"

# Dataset oluştur
dataset = HazeDataset(haze_dict=haze_images_dict, clear_dir=clear_dir, haze_dir=haze_dir, transform=transform)

# DataLoader oluştur
data_loader = DataLoader(dataset, batch_size=8, shuffle=True)


In [48]:
print(f"Number of haze images: {len(haze_images)}")
print(f"Number of clear images: {len(clear_images)}")
# Eşleşen dosya isimlerini bulma
common_images = list(set(haze_images).intersection(set(clear_images)))

# Eşleşen dosyaların sayısını kontrol et
print(f"Number of common images: {len(common_images)}")

# Verilerin eşleşen dosyalarla yüklendiğinden emin olun
haze_images = common_images
clear_images = common_images


Number of haze images: 69165
Number of clear images: 1976
Number of common images: 0


In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader

# Generator (Sisli görüntüden temiz görüntü üretme)
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.output_layer = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.output_layer(x)
        return x

# Discriminator (Gerçek ve sahte görüntüleri ayırt etme)
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(6, 64, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.fc = nn.Linear(256 * 32 * 32, 1)  # Flattening the output and applying a sigmoid
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.conv2(x), 0.2)
        x = F.leaky_relu(self.conv3(x), 0.2)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        x = self.sigmoid(x)  # Output a probability (real or fake)
        return x

# Kaybı ve Optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy

# Optimizer'lar
generator = Generator()
discriminator = Discriminator()
optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# GAN Kaybı
def compute_gan_loss(real_output, fake_output):
    real_loss = criterion(real_output, torch.ones_like(real_output))  # Gerçek görüntüler için 1 etiket
    fake_loss = criterion(fake_output, torch.zeros_like(fake_output))  # Sahte görüntüler için 0 etiket
    return real_loss + fake_loss

# Generator için kayıp
def generator_loss(fake_output):
    return criterion(fake_output, torch.ones_like(fake_output))  # Generator sahteyi gerçek gibi yapmaya çalışır

# Veri Yükleme ve Dataset Yapısı
class HazeDataset(Dataset):
    def __init__(self, haze_images, clear_images, haze_dir, clear_dir, transform=None):
        self.haze_images = haze_images
        self.clear_images = clear_images
        self.haze_dir = haze_dir
        self.clear_dir = clear_dir
        self.transform = transform

    def __len__(self):
        return len(self.haze_images)

    def __getitem__(self, idx):
        haze_img_path = os.path.join(self.haze_dir, self.haze_images[idx])
        clear_img_path = os.path.join(self.clear_dir, self.clear_images[idx])

        haze_image = Image.open(haze_img_path).convert("RGB")
        clear_image = Image.open(clear_img_path).convert("RGB")

        if self.transform:
            haze_image = self.transform(haze_image)
            clear_image = self.transform(clear_image)

        return haze_image, clear_image


# Veri Seti ve DataLoader
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

haze_images = [fname for fname in os.listdir(haze_dir) if fname.endswith(".jpg")]
clear_images = [fname for fname in os.listdir(clear_dir) if fname.endswith(".jpg")]

dataset = HazeDataset(haze_images, clear_images, haze_dir, clear_dir, transform)
data_loader = DataLoader(dataset, batch_size=8, shuffle=True)

# Eğitim Döngüsü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator.to(device)
discriminator.to(device)

num_epochs = 5
for epoch in range(num_epochs):
    for i, (haze_images, clear_images) in enumerate(data_loader):
        haze_images = haze_images.to(device)
        clear_images = clear_images.to(device)

        # Discriminator'ı eğit
        optimizer_d.zero_grad()

        # Gerçek ve sahte görüntülerle discriminator'ı güncelle
        real_output = discriminator(clear_images)
        fake_images = generator(haze_images)
        fake_output = discriminator(fake_images.detach())  # Detach generator, güncellenmesin

        d_loss = compute_gan_loss(real_output, fake_output)
        d_loss.backward()
        optimizer_d.step()

        # Generator'ı eğit
        optimizer_g.zero_grad()

        fake_output = discriminator(fake_images)
        g_loss = generator_loss(fake_output)
        g_loss.backward()
        optimizer_g.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")


IndexError: list index out of range

In [50]:
import os

# Clear ve Haze dosya yolları
clear_dir = '/content/drive/MyDrive/dataset/clear'
haze_dir = '/content/drive/MyDrive/dataset/haze'

# Clear ve Haze dosyalarını alalım
clear_images = [fname for fname in os.listdir(clear_dir) if fname.endswith(".jpg")]
haze_images = [fname for fname in os.listdir(haze_dir) if fname.endswith(".jpg")]

# İlk 10 clear görüntüsünü seçiyoruz
clear_images_subset = clear_images[:10]

# Haze görüntülerini seçiyoruz (clear görüntülerine karşılık gelen tüm haze görüntüleri)
haze_images_dict = {}

for clear_img in clear_images_subset:
    # Clear görüntüsünün temel adını alıyoruz (uzantıyı kaldırıyoruz)
    base_name = os.path.splitext(clear_img)[0]

    # Bu base_name ile başlayan tüm haze görüntülerini buluyoruz
    corresponding_haze = [haze_img for haze_img in haze_images if haze_img.startswith(base_name)]

    # Eğer ilgili haze görüntüleri varsa, bunları sözlüğe ekliyoruz
    if corresponding_haze:
        haze_images_dict[clear_img] = corresponding_haze

# Eşleşen görüntülerin sayısını kontrol edelim
for clear_img, haze_list in haze_images_dict.items():
    print(f"{clear_img} için {len(haze_list)} haze görüntüsü bulundu.")



3824.jpg için 35 haze görüntüsü bulundu.
3588.jpg için 35 haze görüntüsü bulundu.
3414.jpg için 35 haze görüntüsü bulundu.
3680.jpg için 35 haze görüntüsü bulundu.
3762.jpg için 35 haze görüntüsü bulundu.
3525.jpg için 35 haze görüntüsü bulundu.
3511.jpg için 35 haze görüntüsü bulundu.
3845.jpg için 35 haze görüntüsü bulundu.
3764.jpg için 35 haze görüntüsü bulundu.
3412.jpg için 35 haze görüntüsü bulundu.


In [51]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
from torchvision import transforms

class HazeDataset(Dataset):
    def __init__(self, haze_dict, clear_dir, haze_dir, transform=None):
        self.pairs = []  # Clear-Haze çiftlerini burada saklayacağız
        self.clear_dir = clear_dir
        self.haze_dir = haze_dir
        self.transform = transform

        # Sözlükten clear-haze çiftlerini oluştur
        for clear_img, haze_imgs in haze_dict.items():
            for haze_img in haze_imgs:
                self.pairs.append((clear_img, haze_img))

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        clear_img_name, haze_img_name = self.pairs[idx]

        clear_path = os.path.join(self.clear_dir, clear_img_name)
        haze_path = os.path.join(self.haze_dir, haze_img_name)

        clear_image = Image.open(clear_path).convert("RGB")
        haze_image = Image.open(haze_path).convert("RGB")

        if self.transform:
            clear_image = self.transform(clear_image)
            haze_image = self.transform(haze_image)

        return haze_image, clear_image

# Görüntü dönüşümleri (örnek augmentasyon)
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Boyutlandırma
    transforms.ToTensor()          # Tensöre çevirme
])

# Dataset'i oluştur
dataset_subset = HazeDataset(haze_dict=haze_images_dict, clear_dir=clear_dir, haze_dir=haze_dir, transform=transform)

# DataLoader'ı oluştur
data_loader_subset = DataLoader(dataset_subset, batch_size=8, shuffle=True)



In [52]:
import torch.optim as optim
import torch.nn as nn

# Modeli oluştur
model = DynamicCNN()  # Önceden tanımladığınız model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Kayıp fonksiyonu ve optimizasyon
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Eğitim döngüsü
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (haze_images, clear_images) in enumerate(data_loader_subset):
        haze_images = haze_images.to(device)
        clear_images = clear_images.to(device)

        # İleri yayılım
        outputs = model(haze_images)
        loss = criterion(outputs, clear_images)

        # Geri yayılım ve optimizasyon
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(data_loader_subset)}")


Epoch 1/5, Loss: 0.0477844143117016
Epoch 2/5, Loss: 0.014397050063549117
Epoch 3/5, Loss: 0.010384876397438347
Epoch 4/5, Loss: 0.007156380685046315
Epoch 5/5, Loss: 0.006196675695140253
